# Tender2Project
The following Kaggle notebook exploits the long context window of Gemini, in order to fulfill the following targets:
- Analyze technical and commercial tenders for a project.
- Analyze the compatibility of products and solution of certain companies with respect to the tender documentations.
- Find the best company and combination of products and services to build the project, generating a clause by clause report with compliant and not-compliant specifications.

## Notebook structure
The notebook is composed by different parts, each one with a specific target:
- Tenders for a project are parsed, such that their information is converted to text.
- Information scraped from the websites of different companies is loaded as text.
- All the text is forwarded to Gemini in different prompts: combining multi-agent reasoning, chain of toughts and in-chat memory.

## Multi-agent reasoning
Multi-agent reasoning is applied in the code through the segmentation of tasks and the delegation of specific responsibilities to distinct roles. For example:

Technical and Commercial Tender Agents: Separate prompts (tender_prompt_template_technical and tender_prompt_template_commercial) are used to guide the technical tender engineer and the commercial tender manager roles, respectively. Each agent has distinct objectives: identifying and summarizing technical or commercial requirements within tenders. This multi-agent structure ensures detailed and domain-specific analyses.

A distinct prompt is also prepared for analyzing companies (e.g., SIEMENS and HITACHI) to match tender requirements with their products and solutions (get_response_companies_info). This allows tailored reasoning for comparing affinity between tenders and company offerings.

## Chain of Thoughts
The chain of thoughts approach is used to decompose complex tasks into sequential, step-by-step actions, ensuring methodical problem-solving. 
In both technical and commercial prompts we used phrases like "Think step by step" to guide the agent toward incremental reasoning. This ensures that requirements are dissected and analyzed in detail.
The user prompt specifies a structured approach to calculating an affinity score, prompting the agent to explicitly explain the calculation process.
Finally in the Clause-by-Clause Analysis, the final prompt directs the agent to meticulously compare tender requirements with company specifications, maintaining a clear progression in thought.
This approach is embedded in the query processing of tenders and the affinity scoring logic in user_prompt_match and final_prompt, encouraging logical progression in the analysis.

## In-chat memory
The code utilizes in-chat memory to maintain conversational context across multiple interactions. This functionality is facilitated by Chat History Preservation: The function add_history_to_chat appends user queries and model responses (e.g., for tenders or company analyses) to history_chat. This ensures continuity, enabling the model to refer back to previous inputs and outputs during subsequent exchanges.
Additionally, prompts such as system_prompt and user_prompt leverage the accumulated chat history to enhance the depth and relevance of responses. For example, when computing affinity scores or performing a clause-by-clause analysis, the model can reference earlier content in the chat_with_memory object. This allows a continous improvement of the prompt and on the information stored in the chat.



## Conclusion for use case
Using a long context window instead of Retrieval-Augmented Generation (RAG) for the selected use case was particularly beneficial due to the nature of the task, which involves reasoning across interdependent documents, maintaining conversational continuity, and ensuring consistent context for decision-making. 
In fact, in the past, we implemented a multi-agent framework using LangChain and OpenAI, where each company was represented by a dedicated agent. The repo is publicily availbla at https://github.com/SecchiAlessandro/LumadaAI. This framework was designed with a supervisor agent that dynamically routed user queries to the most relevant company-specific agent based on the query context. While innovative for that time, this approach encountered several challenges, particularly in stability, accuracy, and efficiency, making the current solution implemented in the notebook a more effective alternative. 
As the agents are operating independently, it was also difficult to generate combined solutions from different companies.
Moreover, for each query, the supervisor had to perform an additional step of reasoning before invoking an agent.
If the query was relevant to multiple agents, the framework had to perform multiple sequential calls, compounding the latency.

The current solution with a centralized reasoning, ensure consistent application of logic and context.
By avoiding the intermediate step of agent selection, it directly processes queries with a unified context, significantly reducing latency.

The unified context allows the model to cross-reference tender requirements and company offerings directly, ensuring a cohesive and accurate analysis.
This is particularly advantageous for tasks like affinity scoring, which require simultaneous consideration of multiple data points.

The notebook’s approach scales better for handling multiple queries simultaneously, as it avoids the bottleneck of sequential agent calls. In fact, for new tender projects, it is just required to update the in-chat memory and to add new prompts for adding new in-chat agents.


In summary, why we decided to implement tender2project?

1. Holistic Context Retention
Long Context Window: By storing the entire history of tender analyses (both technical and commercial) and company product evaluations, the model retains a comprehensive understanding of all previously provided information. This holistic context allows the model to reason about how specific requirements and offerings interrelate across multiple prompts.
In RAG, the system retrieves only the most relevant chunks of information from the documents for each query. While efficient, this approach can fragment the analysis when tasks require synthesizing insights across documents, potentially leading to overlooked interconnections.
2. Interdependent Analysis
Tender and Company Matching: The task involves comparing multiple tenders against products and solutions offered by different companies, followed by calculating an affinity score and conducting a clause-by-clause compliance analysis. These steps require information from previous steps to be accessible and integrated seamlessly.
RAG typically retrieves context independently for each query, which might result in loss of nuance or context-dependent reasoning, especially when relationships between multiple documents must be preserved. A long context window ensures the model has immediate access to the entire conversational flow and insights developed so far.
3. Dynamic Multi-Agent Collaboration
Role-Based Prompts: By maintaining a long context, the system can simulate multi-agent collaboration, where outputs from technical engineers, commercial managers, and sales managers flow into a unified reasoning framework.
In RAG, each role’s analysis would require re-retrieving relevant information from documents, potentially leading to inconsistencies or duplications. With a long context window, outputs from one role naturally inform others, creating a seamless chain of thought.
4. Reduced Query Overhead
Fewer Retrievals, Continuous Focus: Long context windows reduce the need for multiple retrieval calls, making the process more efficient in scenarios where information is revisited or refined iteratively.
RAG introduces latency and computational costs because each query requires searching and ranking document chunks. A long context window allows for continuous focus on the task, with all prior exchanges readily available.
5. Affinity Score Calculation
Global Context for Consistent Metrics: Computing an affinity score across companies for tenders requires integrating technical and commercial analysis alongside company data. This step benefits significantly from the model’s ability to access all previous responses simultaneously.
In RAG, affinity scoring would require separate retrievals of technical requirements, commercial requirements, and company data for each tender. This could introduce discrepancies if context for one query is inadvertently excluded during retrieval.
6. Clause-by-Clause Compliance Analysis
Integrated Insight Application: Clause-by-clause analysis relies on cross-referencing previously extracted requirements with company offerings. The long context window allows the model to directly reference earlier inputs and outputs without reloading or retrieving.
RAG retrievals for clause-by-clause analysis might lead to inconsistencies if prior reasoning is split across multiple retrievals. A long context window ensures the model "remembers" and applies earlier analyses cohesively.
Trade-offs and Use Case Fit

## Conclusion
The centralized, long context window approach provides clear advantages in stability, response time, and accuracy over the earlier multi-agent framework. It highlights the importance of selecting a system architecture that aligns with the specific demands of the use case, particularly for complex, multi-faceted analyses like those in tender evaluations, clause-by-clause generation and company affinity scoring.
#### The long context window acts as a shared workspace, recording and making all agent outputs accessible for seamless and holistic reasoning. In today's interconnected world, where partnerships and synergies are essential to addressing complex challenges, we envision a tool that enables continuous reasoning, uncovers new patterns and solutions, and minimizes the fragmentation of insights.




Clean the working directory

In [ ]:
! rm -r /kaggle/working/*

In [ ]:
from IPython.display import Markdown

# Load the tenders

Locate the tenders - in PDF format.

In [ ]:
# fetch the script to download content from GitHub
!wget https://raw.githubusercontent.com/gabripo/kaggle-gemini-long-context/refs/heads/main/github_downloader.py -P /kaggle/working/scripts

# add downloaded script to the Python path
import sys
sys.path.append('/kaggle/working/scripts')

In [ ]:
import github_downloader

github_downloader.download_files_from_github_repo(folderName="tenders", saveFolder="/kaggle/working/tenders", extension="pdf")

In [ ]:
import os

# if the PDF file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_tender = False if os.path.exists('/kaggle/working/tenders') else True
if use_kaggle_input_tender:
    tenders_file_path = '/kaggle/input/tenders'
else:
    tenders_file_path = '/kaggle/working/tenders'

print(f"The folder {tenders_file_path} will be considered as containing the tenders")

Installing required Python packages to analyze the tender - in PDF format.

In [ ]:
!pip install PyPDF2

Extract information from the tenders.
The output will be a text.

In [ ]:
import os
from PyPDF2 import PdfReader

tenders = [t for t in os.listdir(tenders_file_path) if t.endswith(".pdf")]
tenders_info = {}
for tender in tenders:
    print(f"Reading the tender {tender} ...")
    reader = PdfReader(os.path.join(tenders_file_path, tender))

    tenders_info[tender] = {}
    tenders_info[tender]["name"] = tender
    tenders_info[tender]["content"] = "\n".join([page.extract_text() for page in reader.pages])
#Markdown(tenders_info["tender_solar.pdf"]["content"])
# information for each tender can be accessed by:
#tenders_info["tender_solar.pdf"]["content"]

# Fetch information about companies

## Overview
Information about interesting companies is obtained from their websites.

To generate data out of the companies' websites, we implemented a crawler.
The final output of the crawler is a JSON file, in which each field refers to a company: for each company, all the information of the websites is merged.

> To make things easier, the mentioned JSON file will be fetched from a Git repository where the crawling function has already been executed.

## Details about the crawling process:
- **Recursive scan**: after a webpage is scanned and its content is stored, eventual found sublinks are scanned, as well. A limit of the wepages to download is given as input.
- **Redundant information is deleted**: if some website content can be found multiple times in all the webpages of one company, then it is skipped. *Example*: undesired and redundant lines like "Contact Us" are removed, ensuring that the final content does not include unnecessary sentences.
- **Caching of already downloaded pages**: for each webpage, the content is stored in a JSON file, as well as the found sublinks. *Example*: after a run with a limit of N pages, other runs with less than N pages will use the stored files instead downloading data from internet; at the contrary, if the limit is increased to M > N pages, only M - N additional pages will be downloaded while the first N pages will be taken from the stored file.

## Load the results from the crawler's repo

In [ ]:
github_downloader.download_files_from_github_repo(folderName="", saveFolder="/kaggle/working/companies_info", extension="json")

Locate the JSON file containing the companies' information.

In [ ]:
import os

# if the JSON file is given as Kaggle Input (for example, manually uploaded), change the use_kaggle_input to True
use_kaggle_input_companies = False if os.path.exists('/kaggle/working/companies_info') else True
companies_json_name = 'companies_info.json'
if use_kaggle_input_companies:
    companies_info_file_path = os.path.join('/kaggle/input/companies-info', companies_json_name)
else:
    companies_info_file_path = os.path.join('/kaggle/working/companies_info', companies_json_name)

print(f"The file {companies_info_file_path} will be used for the information regarding the companies")

Define a small function to read the information about the companies - in JSON format.

In [ ]:
import json

def read_json_info(jsonFilePath: str) -> dict:
    if os.path.exists(jsonFilePath):
        with open(jsonFilePath, "r") as f:
            data = json.load(f)
        return data
    else:
        return {}

Load the companies' information by using the defined function.

In [ ]:
companies_info = read_json_info(companies_info_file_path)

# companies_info is a dictionary, where the key is the name of the company and the related value its information
# print(companies_info["SIEMENS"]) 

# Chat with Gemini

In [ ]:
# API key got here: https://ai.google.dev/tutorials/setup

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_key = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key = secret_key)

model_name = 'gemini-1.5-flash-latest'
model = genai.GenerativeModel(model_name=model_name)

chat = model.start_chat()

model_info = genai.get_model(f"models/{model_name}")
print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")

## Analyze all the tenders

In [ ]:
download_tenders_json_from_repo = False # switch to False if willing to generate the json file within this notebook

if download_tenders_json_from_repo:
    # this helps reducing the Gemini Quota, since no queries will be performed for the tenders
    github_downloader.download_files_from_github_repo(folderName="tenders", saveFolder=tenders_file_path, extension="json")

In [ ]:
tender_prompt_template_technical = """
You are an experienced technical tender engineer. 
The document you have is a tender, that contains also technical requirements for a project.
Think step by step on how to look for the relevant technical requirements and make a detailed summary.
The content of the document is: """
tender_prompts_technical = []
for info in tenders_info.values():
    tender_prompts_technical.append(f"You have a document called {info['name']} . " + tender_prompt_template_technical + f"{info['content']}")

In [ ]:
tender_prompt_template_commercial = """
You are an experienced commercial tender manager. 
The document you have is a tender, that contains also commercial requirements for a project.
Think step by step on how to look for the relevant commercial requirements and make a detailed summary.
The content of the document is: "
"""
tender_prompts_commercial = []
for info in tenders_info.values():
    tender_prompts_commercial.append(f"You have a document called {info['name']} . " + tender_prompt_template_commercial + f"{info['content']}")

In [ ]:
from time import sleep

def get_responses_tenders(subject, tender_prompts):

    responses = {}
    tenders_json_file_path = os.path.join(tenders_file_path, f'tenders_{subject}.json')
    if os.path.exists(tenders_json_file_path):
        responses = read_json_info(tenders_json_file_path)
        print(f"Responses loaded from file {tenders_json_file_path}")
    else:
        num_queries = 0
        for tender_prompt, tender_name in zip(tender_prompts, tenders):
            print(f"Generating response for tender {tender_name} ...")
            response = chat.send_message(tender_prompt)
            # print(response.text)
            responses[tender_name] = {'prompt': tender_prompt, 'answer': response.text}
            print(f"Response for tender {tender_name} generated.")
    
            num_queries += 1
            """
            if num_queries % 2 == 0:
                wait_time_seconds = 90
                print(f"Waiting {wait_time_seconds} before continuing, to not exceed Gemini's quota")
                sleep(wait_time_seconds)
            """
    
        with open(tenders_json_file_path, 'w') as f:
            json.dump(responses, f, ensure_ascii=True, indent=4)
        print(f"Responses stored into {tenders_json_file_path}")
    
    print(f"tender_{subject}: Analysis concluded!")
    return responses
    
response_technical = get_responses_tenders("technical", tender_prompts_technical)
response_commercial = get_responses_tenders("commercial", tender_prompts_commercial)

In [ ]:
def get_response_companies_info(company_name):
    
    responses = {}
    company_prompt = f"these are the information of products and solutions for the company {company_name} : {companies_info[company_name]}"
    response = chat.send_message(company_prompt)
    responses[company_name] = {'prompt': company_prompt, 'answer': response.text}

    print(f"response for company {company_name} generated!")
    return responses


In [ ]:
response_hitachi = get_response_companies_info("HITACHI")

In [ ]:
import time
time.sleep(60)

In [ ]:
response_siemens = get_response_companies_info("SIEMENS")

In [ ]:
# example how to include the chat history here https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb
# description of the Content class here https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md
from google.generativeai.protos import Content, Part

history_chat = []

def add_history_to_chat(responses, user, history_chat):
    for idx_response, response in enumerate(responses.values()):
        query = Part()
        query.text = f"{user}: {response['prompt']}"
        # TODO: consider different users? Example: for the tender number idx_response, use f"user_{idx_response}"
        history_chat.append(Content(role="user", parts=[query]))
    
        answer = Part()
        answer.text = response['answer']
        history_chat.append(Content(role="model", parts=[answer]))
    return 

add_history_to_chat(response_technical, "technical engineer", history_chat)
add_history_to_chat(response_commercial, "commercial manager", history_chat)
add_history_to_chat(response_siemens, "sales manager for siemens", history_chat)
add_history_to_chat(response_hitachi, "sales manager for hitachi", history_chat)


## Define the system and user prompts

In [ ]:
system_prompt = """
You are an experienced team of business development managers and tender engineers, commercial managers.
You need to create a detailed clause by clause from the tender documentations and the most affine company specifications.
"""

Markdown(system_prompt)

In [ ]:
user_prompt_match = """

1. For company SIEMENS and HITACHI, find the respective relevant products and solutions with respect to the analyzed tenders.
   The information is in the form of text I provided, then you do not need to read additional documents or access to websites.
   
   
2. Calculate an affinity score in percentage for each company based on analysis in point 1. Explain the way how you computed this percentage.

"""

# Markdown(user_prompt)

## Test the history

In [ ]:
chat_with_memory = model.start_chat(history=history_chat)
response = chat_with_memory.send_message("Which are the roles given in the prompt from the user? There are only two for tenders and one for company")

Markdown(response.text)

In [ ]:
#response = chat_with_memory.send_message("Detailed explanation of the technical specifications of the tenders")

#Markdown(response.text)

## Count the needed tokens

## Generate the final response

In [ ]:
print("Finding the most suitable company for the tenders ...")
match_prompt = f"{user_prompt_match}"
match_response = chat_with_memory.send_message(match_prompt)
# print(response.text)

print("Response to the prompts is ready!")

In [ ]:
Markdown(match_response.text)

In [ ]:
user_prompt = """

Consider the company with the highest affinity score 
and return the clause by clause analysis considering technical and commercial compliant and not-compliant requirements
of the tender with respect to the selected company. Report also the URL of the source where you found the informations.

"""

# Markdown(user_prompt)

In [ ]:
print(f"{model.count_tokens(history_chat)=}")
print(f"{model.count_tokens(system_prompt)=}")
print(f"{model.count_tokens(user_prompt)=}")

In [ ]:
print("Generating the clause by clause ...")
final_prompt = f"{system_prompt}\n\n{user_prompt}"
final_response = chat_with_memory.send_message(final_prompt)
# print(response.text)

print("Response to the prompts is ready!")

In [ ]:
Markdown(final_response.text)